In [2]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import sys
from pathlib import Path
proj_path = Path('.').resolve()
sys.path.append(str(proj_path))

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain_chroma import Chroma
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

import json
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd
from typing import Optional
from collections import defaultdict
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.runnables import RunnableSequence
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

_ = load_dotenv(find_dotenv())

from src.db_utils import get_schema_str, get_db_file
from src.pymodels import (
    DatabaseModel, 
    SpiderSample, 
    BirdSample, 
    BODescription,
    SQLResponse,
    KeywordExtractionResponse,
    GenTemplateResponse
)
from src.prompts import Prompts
from src.database import SqliteDatabase
from src.data_preprocess import (
    load_raw_data,
    process_all_tables,
    filter_samples_by_count_spider_bird,
    process_samples_bird,
    split_train_dev_test,
    save_samples_spider_bird,
    load_samples_spider_bird,
)
from src.eval_utils import (
    get_complexity, 
    run_sqls_parallel,
    get_all_structural_score,
    get_all_semantic_score,
    # run_sqls,
    execute_model,
    SKIP_DB_IDS
)

from src.parsing_sql import (
    Schema, extract_all, extract_aliases, _format_expression, STRING_TYPE, NUMERIC_TYPE
)
from run_bo_sql import get_vector_store
from copy import deepcopy

from src.eval_utils import get_structural_score, get_all_structural_score, get_all_semantic_score, partial_matching_with_penalty
from run_evaluation import get_target_parsed_sql, get_prediction_parsed_sql
from run_bo_sql import batched, _get_categories, _format_interval, get_retriever
from bert_score import score as bscore
from transformers import logging as tfloggings
tfloggings.set_verbosity_error()
import warnings

/home/simonjisu/code/BusinessObjects/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/simonjisu/code/BusinessObjects/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# from langchain.globals import set_llm_cache
# from langchain_community.cache import SQLiteCache
# set_llm_cache(SQLiteCache(database_path=f"./cache/valid_bo_bird_dev.db"))

In [ ]:
from pprint import pprint
def get_content(x: dict):
    return json.loads(x['kwargs']['message']['kwargs']['content'])
task = 'direct'  
# task = 'keyword_extraction'
db_id = 'movie_3'
prefix = 'x-'
file_name = f'{task}_{db_id}'
db = SqliteDatabase(f"./cache/{prefix}{file_name}.db")

# df = db.execute("SELECT * FROM full_llm_cache WHERE ROWID = (SELECT MAX(ROWID) FROM full_llm_cache);")
df = db.execute("SELECT ROWID, response FROM full_llm_cache;")
df.shape

In [17]:
content = get_content(json.loads(df['response'].iloc[-3]))
pprint(content)

{'hint_used': False,
 'rationale': ['Identify the customer by name: We need to find the customer '
               "'Marcelia Goering' in the 'customer' table to get the "
               'customer_id.',
               'Join the necessary tables: To find the orders placed by this '
               "customer, we need to join the 'customer', 'cust_order', and "
               "'order_line' tables.",
               'Filter by year: We need to filter the orders that were placed '
               'in the year 2021 by checking the order_date in the '
               "'cust_order' table.",
               'Filter by shipping method: We also need to ensure that the '
               "shipping method used is 'Priority Shipping' by joining with "
               "the 'shipping_method' table."],
 'sql': 'SELECT COUNT(*) FROM cust_order AS co INNER JOIN customer AS c ON '
        'co.customer_id = c.customer_id INNER JOIN shipping_method AS sm ON '
        "co.shipping_method_id = sm.method_id WHERE c.fir

In [18]:
x = '''To return latitude and longitude along with station details, I will select the lat and long columns from the station table.
'''.strip()
df.loc[df['response'].str.contains(x)]

,rowid,response
599,601,"{""lc"": 1, ""type"": ""constructor"", ""id"": [""langc..."


In [19]:
# idxes = df['response'].apply(lambda x: True if get_content(json.loads(x)).get(' ') else False)
# idxes = df['response'].str.contains("google_entity_id_id")
# idxes = ','.join(df.loc[idxes, 'rowid'].astype(str).tolist())
# idxes
idxes = ','.join(df.loc[df['response'].str.contains(x)].loc[:, 'rowid'].astype(str).values.tolist())
idxes

'601'

In [20]:
file_name = f'{task}_{db_id}'
db = SqliteDatabase(f"./cache/{prefix}{file_name}.db")
db.start()
c = db.con.cursor()
c.execute('BEGIN TRANSACTION')

# remove the last row record
c.execute(f"""
DELETE FROM full_llm_cache
WHERE ROWID IN ({idxes});
""")
db.con.commit()
db.close()
# (SELECT MAX(ROWID) FROM full_llm_cache)
# JSONDecodeError
# ValidationError
# c.execute("""
# DELETE FROM full_llm_cache WHERE response LIKE '%JSONDecodeError%';
# """)

In [ ]:
from pprint import pprint
df = db.execute(
'''
SELECT * FROM full_llm_cache
WHERE prompt LIKE '%Which teams have had a player awarded the Purple Cap and another with the Orange Cap%'
''')
df

---

In [ ]:
from src.eval_utils import get_structural_score, get_all_structural_score, get_all_semantic_score, partial_matching_with_penalty
from run_evaluation import get_target_parsed_sql, get_prediction_parsed_sql
from run_bo_sql import _get_categories, _format_interval, get_retriever
from bert_score import score as bscore
from transformers import logging as tfloggings
tfloggings.set_verbosity_error()
import warnings

ds = 'spider'
task = 'zero_shot_hint'
typ = 'test'
scenario = 0
description_file = f'description.json' if ds == 'spider' else f'{ds}_description.json'
experiment_folder = proj_path / 'experiments' / ds
prediction_path = experiment_folder / 'predictions' / task
eval_path = experiment_folder / 'evals' / task

tables, *_ = load_raw_data(proj_path / 'data' / ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

In [ ]:
# bird_path = proj_path / 'data' / 'bird'
# tables, train_data, dev_data = load_raw_data(bird_path, load_test=False)

# with (proj_path / 'data' / 'bird_description.json').open() as f:
#     all_descriptions = json.load(f)

# bird_tables = process_all_tables(tables, descriptions=all_descriptions)
# train_samples = load_samples_spider_bird(proj_path / 'data' / 'bird_train.json')
# dev_samples = load_samples_spider_bird(proj_path / 'data' / 'bird_dev.json')
# test_samples = load_samples_spider_bird(proj_path / 'data' / 'bird_test.json')

In [ ]:
# file_post_fix = f'{ds}_{typ}' if scenario < 0 else f'{ds}_{typ}_{scenario}'
# final_file = f'final_{file_post_fix}.json'
# samples = load_samples_spider_bird(proj_path / 'data' / f'{ds}_{typ}.json')

# if not (prediction_path / final_file).exists():
#     all_results = []
#     paths = sorted(list(prediction_path.glob(f'{file_post_fix}_*.json')))
#     for p in paths:
#         with p.open() as f:
#             results = json.load(f)
            
#         for r in results:
#             r.pop('rationale')
#             r['db_id'] = p.stem.split('_', 3)[-1]

#             found = False
#             for s in samples:
#                 if r['sample_id'] == s.sample_id:
#                     found = True
#                     break
#             r['gold_sql'] = s.final.sql
#             assert found, r['sample_id']

#         all_results.extend(results)

In [ ]:
# with open(prediction_path / final_file, 'r') as f:
#     preds = json.load(f)

In [ ]:
# pred_parsed, _ = get_prediction_parsed_sql(preds, tables)

# Training a retrieval model

In [3]:
ds = 'bird'
typ = 'dev'
task = 'retrieval' # 'zero_shot', 'retrieval', 'valid_bo'
description_file = f'description.json' if ds == 'spider' else f'{ds}_description.json'
experiment_folder = proj_path / 'experiments' / ds

eval_path = experiment_folder / 'evals' / task
prediction_path = experiment_folder / 'predictions' / task

tables, *_ = load_raw_data(proj_path / 'data' / ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

samples = load_samples_spider_bird(proj_path / 'data' / f'{ds}_{typ}.json')

In [ ]:
# with open(experiment_folder / 'predictions' / 'create_bo' / f'final_{ds}_train_bo.json', 'r') as f:
#     train_bo = json.load(f)

# df = []
# for db_id, xs in train_bo.items():
#     for x in xs:
#         x['db_id'] = db_id
#         df.append(x)

# df = pd.DataFrame(df)
# cates = pd.qcut(df['gold_complexity'], q=5)
# df['gold_complexity_cates'] = cates
# df['gold_complexity_codes'] = cates.cat.codes
# df.to_csv(experiment_folder / 'predictions' / 'create_bo' / f'final_{ds}_train_bo.csv', index=False)
# df = pd.read_csv(experiment_folder / 'predictions' / 'create_bo' / f'final_{ds}_train_bo.csv')

# df_train = df.groupby('gold_complexity_codes').sample(frac=0.9, random_state=42)
# df_dev = df.drop(df_train.index)

# print(df_train['gold_complexity_codes'].value_counts().sort_index())
# print(df_dev['gold_complexity_codes'].value_counts().sort_index())

# df_train.to_csv(experiment_folder / 'predictions' / 'create_bo' / f'{ds}_{task}_trainset.csv', index=False)
# df_dev.to_csv(experiment_folder / 'predictions' / 'create_bo' / f'{ds}_{task}_devset.csv', index=False)

# Procedures

## task: valid_bo

1. (retrieve) retrieve and rerank for dev dataset to get relevant train BOs
2. (gen_template) generate sql template with dev dataset and retrieved BOs --> output: dev sql templates
3. (keyword_extraction) extract keywords from dev sql templates --> output: keywords
4. (search_value) search for the most helpful values for the dev sql templates --> output: columns with values
5. (fill_in) fill-in values with dev sql templates --> output: dev sql
6. (evaluate) evaluate the dev sql to get the most helpful train BOs --> output: BOs Pool

## task: inference_with_bo

1. (retrieve) retrieve and rerank for test dataset from BOs Pool
2. (gen_template) generate sql template with test dataset and retrieved BOs --> output: test sql templates
3. (keyword_extraction) extract keywords from dev sql templates --> output: keywords
4. (search_value) search for the most helpful values for the dev sql templates --> output: columns with values
5. (fill_in) fill-in values with test sql templates --> output: test sql
6. (evaluate) evaluate the test sql


### evaluate

In [2]:
class ARGS():
    exp_name = 'pipeline_exp' # 'pipeline_exp'
    ds = 'bird'
    type = 'dev'
    task = 'evaluate' 
    eval_target = 'fill_in' # 'fill_in'
    with_bos = False
    
args = ARGS()

description_file = f'description.json' if args.ds == 'spider' else f'{args.ds}_description.json'
experiment_folder = proj_path / args.exp_name / args.ds
eval_path = experiment_folder / 'evals'

description_file = 'bird_description.json' if args.ds == 'bird' else 'description.json'
tables, *_ = load_raw_data(proj_path / 'data' / args.ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

def parse_sql_to_output(sql: str, schema: Schema):
    try:
        ei = extract_all(sql, schema)
        assert len(ei['sel']) > 0, f'No selection found'
    except Exception as e:
        return None
    return ei

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')

file_name = f'{"with_bos" if args.with_bos else "no_bos"}-{args.type}'
eval_target_path = experiment_folder / 'predictions' / args.eval_target / f'{file_name}.json'
assert eval_target_path.exists(), f'Run with the `task={args.eval_target}, type={args.type}` first'
with eval_target_path.open() as f:
    eval_targets = json.load(f)

# TODO: for loop should be start from pred_queries

In [5]:
import hashlib

In [6]:
parsed_queries_path: Path = eval_path / f'parsed_queries-{file_name}.pkl'
parsed_queries_exist = parsed_queries_path.exists()
if parsed_queries_exist:
    with parsed_queries_path.open('rb') as f:
        parsed_queries = pickle.load(f)
else:
    parsed_queries = defaultdict(dict)
    parsed_queries['unable'] = set()

samples_by_id = {s.sample_id: s for s in samples}
eval_data: dict[str, list] = defaultdict(list)
eval_data2doc_ids = defaultdict(set)

# from flatten to nested by doc_ids
for pred in tqdm(eval_targets, total=len(eval_targets), desc='preparing eval data'):
    sample_id = pred['sample_id']
    pred_sql = pred['sql']
    doc_ids = pred['doc_ids']
    target_sample = samples_by_id[sample_id]
    target_sql = target_sample.final.sql
    db_id = target_sample.db_id
    key = hashlib.sha256(f'{sample_id}-{pred_sql}'.encode()).hexdigest()

    if not parsed_queries_exist:
        schema = Schema(tables[db_id].db_schema)
        if parsed_queries['target'].get(target_sql) is None:
            target_output = parse_sql_to_output(target_sql, schema)
            parsed_queries['target'][target_sql] = target_output
        else:
            target_output = parsed_queries['target'][target_sql]
        if parsed_queries['pred'].get(pred_sql) is None:
            pred_output = parse_sql_to_output(pred_sql, schema)
            parsed_queries['pred'][pred_sql] = pred_output
        else:
            pred_output = parsed_queries['pred'][pred_sql]
    
        if (not pred_output) or (not target_output):
            if key not in parsed_queries['unable']:
                parsed_queries['unable'].add(key)
            continue
    else:
        if key in parsed_queries['unable']:
            continue
    
    if key not in eval_data2doc_ids:
        eval_data['sample_ids'].append(sample_id)
        eval_data['target_queries'].append(target_sql)
        eval_data['db_paths'].append(get_db_file(proj_path, args.ds, db_id))
        eval_data['pred_queries'].append(pred_sql)
        eval_data2doc_ids[key].update(doc_ids)

    for doc_id in doc_ids:
        eval_data2doc_ids[key].add(doc_id)

if not parsed_queries_exist:
    with parsed_queries_path.open('wb') as f:
        pickle.dump(parsed_queries, f)

preparing eval data: 100%|██████████| 56912/56912 [00:02<00:00, 22534.92it/s]


In [ ]:
i = 0
sample_id = eval_data['sample_ids'][i]
target_query = eval_data['target_queries'][i]
db_file = eval_data['db_paths'][i]
pred_query = eval_data['pred_queries'][i]
db = SqliteDatabase(db_file)

print(sample_id)
print(target_query)
print(db_file)
print(pred_query)
result = execute_model(pred_query, target_query, db_file, sample_id, 30.0)
result

2136
SELECT CAST(SUM(CASE WHEN T2.value > 50 THEN 1 ELSE 0 END) AS REAL) * 100 / COUNT(T1.CountryCode) FROM Country AS T1 INNER JOIN Indicators AS T2 ON T1.CountryCode = T2.CountryCode WHERE T1.Region = 'South Asia' AND T2.IndicatorName = 'Life expectancy at birth, female (years)'
/home/simonjisu/code/BusinessObjects/data/bird/train/train_databases/world_development_indicators/world_development_indicators.sqlite
SELECT (COUNT(CASE WHEN indicators.value > 50 THEN 1 END) * 100.0 / COUNT(*)) AS percentage FROM country INNER JOIN indicators ON country.countrycode = indicators.countrycode WHERE country.region = 'South Asia' AND indicators.indicatorname = 'Life expectancy at birth, female (years)'


In [7]:
from itertools import product, islice
def batched(iterable, n, *, strict=False):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch

n_batch = 2000
n_samples = len(eval_data['sample_ids'])
batches = list(batched(range(n_samples), n_batch))

In [8]:
batch_i = 7
idxes = batches[batch_i]
batch_results = []
batch_eval_data = {k: [v[i] for i in idxes] for k, v in eval_data.items()}
batch_preds = [x for x in batch_eval_data['pred_queries']]
batch_sample_ids = [x for x in batch_eval_data['sample_ids']]

In [9]:
sample_ids = batch_eval_data['sample_ids']
pred_queries = batch_eval_data['pred_queries']
target_queries = batch_eval_data['target_queries']
db_paths = batch_eval_data['db_paths']
exec_result = [None] * len(sample_ids)

samples_by_db = defaultdict(list)
for i, (sample_id, pred, target, db_file) in enumerate(zip(sample_ids, pred_queries, target_queries, db_paths)):
    samples_by_db[db_file].append((i, sample_id, pred, target))

# for db_file, samples in samples_by_db.items():
#     for i, sample_id, pred, target in tqdm(
#         samples, total=len(samples), desc=f"Processing {Path(db_file).stem}"
#     ):
        # if Path(db_file).stem in SKIP_DB_IDS:
        #     result = {"sample_id": sample_id, "res": 0, "target_error": False}
        # else:
        #     result = execute_model(pred, target, db_file, sample_id, 30.0)
        # exec_result[i] = result

In [ ]:
sid = 10904

sample_id = list(filter(lambda x: x == sid, sample_ids))[0]
pred = pred_queries[i]
target = target_queries[i]
db_file = db_paths[i]

target = target.replace(';', '')
target_sql = f'SELECT * FROM ({target}) LIMIT {10};'
pred = pred.replace(';', '')
pred_sql = f'SELECT * FROM ({pred}) LIMIT {10};'
db = SqliteDatabase(db_file)

db.execute(pred_sql)

,COUNT(product.productid)
0,0


In [ ]:
sid = 10904

idxes = list(filter(lambda x: x[1] == sid, enumerate(sample_ids)))
for i, _ in idxes:
    print(i)
    pred = pred_queries[i]
    target = target_queries[i]
    db_file = db_paths[i]

    target = target.replace(';', '')
    target_sql = f'SELECT * FROM ( {target} ) LIMIT {10};'
    pred = pred.replace(';', '')
    pred_sql = f'SELECT * FROM ( {pred} ) LIMIT {10};'
    db = SqliteDatabase(db_file)

    db.execute(pred_sql)
    break

In [ ]:
"""
SELECT 
    customers.segment, (
    (SUM(CASE WHEN yearmonth.date = '2013-08-23' 
    THEN yearmonth.consumption ELSE 0 END) - SUM(CASE WHEN yearmonth.date = '2012-08-23' 
    THEN yearmonth.consumption ELSE 0 END)) / NULLIF(SUM(CASE WHEN yearmonth.date = '2012-08-23' 
    THEN yearmonth.consumption ELSE 0 END), 0)) * 100 AS percentage_increase 
    FROM customers 
    INNER JOIN yearmonth 
    ON customers.customerid = yearmonth.customerid 
    WHERE yearmonth.date IN ('2012-08-23', '2013-08-23') AND customers.segment IN ('SME', 'LAM', 'KAM') 
    GROUP BY customers.segment ORDER BY percentage_increase DESC LIMIT 1 SELECT customers.segment, ((SUM(CASE WHEN yearmonth.date = '2013-08-23' THEN yearmonth.consumption ELSE 0 END) - SUM(CASE WHEN yearmonth.date = '2012-08-23' THEN yearmonth.consumption ELSE 0 END)) / NULLIF(SUM(CASE WHEN yearmonth.date = '2012-08-23' THEN yearmonth.consumption ELSE 0 END), 0)) * 100 AS percentage_increase FROM customers INNER JOIN yearmonth ON customers.customerid = yearmonth.customerid WHERE yearmonth.date IN ('2012-08-23', '2013-08-23') AND customers.segment IN ('SME', 'LAM', 'KAM') GROUP BY customers.segment ORDER BY percentage_increase ASC LIMIT 1"""

"SELECT customers.segment, ((SUM(CASE WHEN yearmonth.date = '2013-08-23' THEN yearmonth.consumption ELSE 0 END) - SUM(CASE WHEN yearmonth.date = '2012-08-23' THEN yearmonth.consumption ELSE 0 END)) / NULLIF(SUM(CASE WHEN yearmonth.date = '2012-08-23' THEN yearmonth.consumption ELSE 0 END), 0)) * 100 AS percentage_increase FROM customers INNER JOIN yearmonth ON customers.customerid = yearmonth.customerid WHERE yearmonth.date IN ('2012-08-23', '2013-08-23') AND customers.segment IN ('SME', 'LAM', 'KAM') GROUP BY customers.segment ORDER BY percentage_increase DESC LIMIT 1 SELECT customers.segment, ((SUM(CASE WHEN yearmonth.date = '2013-08-23' THEN yearmonth.consumption ELSE 0 END) - SUM(CASE WHEN yearmonth.date = '2012-08-23' THEN yearmonth.consumption ELSE 0 END)) / NULLIF(SUM(CASE WHEN yearmonth.date = '2012-08-23' THEN yearmonth.consumption ELSE 0 END), 0)) * 100 AS percentage_increase FROM customers INNER JOIN yearmonth ON customers.customerid = yearmonth.customerid WHERE yearmonth.da

### aggregate

In [11]:
class ARGS():
    exp_name = 'pipeline_exp'# direct_exp, pipeline_exp
    ds = 'bird'
    type = 'dev'
    task = 'evaluate'
    eval_target = 'fill_in'  # direct, fill_in
    with_bos = False
    
args = ARGS()

description_file = f'description.json' if args.ds == 'spider' else f'{args.ds}_description.json'
experiment_folder = proj_path / args.exp_name / args.ds
eval_path = experiment_folder / 'evals'

description_file = 'bird_description.json' if args.ds == 'bird' else 'description.json'
tables, *_ = load_raw_data(proj_path / 'data' / args.ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')
# drop samples that are in SKIP_DB_IDS
# samples = [s for s in samples if s.db_id not in SKIP_DB_IDS]

file_name = f'{"with_bos" if args.with_bos else "no_bos"}-{args.type}'
samples_by_id = {s.sample_id: s for s in samples}

if args.eval_target == 'fill_in':  # pipeline method
    # gen_template
    with open(experiment_folder / 'predictions' / 'gen_template' / f'{file_name}.json', 'r') as f:
        sql_templates = json.load(f)

    # keyword_extraction
    with open(experiment_folder / 'predictions' / 'keyword_extraction' / f'{file_name}.json', 'r') as f:
        keyword_extraction = json.load(f)

    # search_value
    with open(experiment_folder / 'predictions' / 'search_value' / f'{file_name}.json', 'r') as f:
        search_value = json.load(f)
else:
    sql_templates = []
    keyword_extraction = []
    search_value = []

with open(experiment_folder / 'evals' / f'execution_result-{file_name}.json', 'r') as f:
    exec_results = json.load(f)

with open(experiment_folder / 'evals' / f'merit_result-{file_name}.json', 'r') as f:
    merit_results = json.load(f)

# print(len(exec_results))
# skip_ids = [s.sample_id for s in samples if s.db_id in SKIP_DB_IDS]
# exec_results = [r for r in exec_results if r['sample_id'] not in skip_ids]
# merit_results = [r for r in merit_results if r['sample_id'] not in skip_ids]
# samples_by_id = {s.sample_id: s for s in samples}
# print(len(exec_results))

all_results = defaultdict(list)
# column: sample_id, db_id, retrieved, exec_res, structural_score, semantic_score, f1_score, target_complexity
# if fill_in: bo_used, keywords, values
for i in range(len(exec_results)):
    sample_id = exec_results[i]['sample_id']
    doc_ids = exec_results[i]['doc_ids']
    exec_res = exec_results[i]['res']
    structural_score = merit_results[i]['structural_score']
    semantic_score = merit_results[i]['semantic_score']
    f1_score = merit_results[i]['f1_score']
    target_complexity = merit_results[i]['target_complexity']
    sample = samples_by_id[sample_id]
    db_id = sample.db_id

    all_results['sample_id'].append(sample_id)
    all_results['db_id'].append(db_id)
    all_results['retrieved'].append(','.join(map(str, doc_ids)))
    all_results['exec_res'].append(exec_res)
    all_results['structural_score'].append(structural_score)
    all_results['semantic_score'].append(semantic_score)
    all_results['f1_score'].append(f1_score)
    all_results['target_complexity'].append(target_complexity)
    
    if args.eval_target == 'fill_in':
        if args.with_bos:
            bo_used = int(sql_templates[i]['hint_used'])
            all_results['bo_used'].append(bo_used)
        keywords = json.dumps({column_name: kws for column_name, kws in keyword_extraction[i]['keywords'].items() if kws})
        values = json.dumps(search_value[i]['values'])
        all_results['keywords'].append(keywords)
        all_results['values'].append(values)

df = pd.DataFrame(all_results)

In [12]:
bo_path = experiment_folder / 'predictions' / 'create_bo' / f'final_{args.ds}_train_bo.json'
with bo_path.open() as f:
    bos = json.load(f)

In [13]:
sample_id_retrieved = df.groupby(['db_id', 'sample_id'])['retrieved'].apply(set).reset_index()
for i, row in sample_id_retrieved.iterrows():
    db_id = row['db_id']
    sample_id = row['sample_id']
    retrieved = row['retrieved']
    bo_ids = [x['sample_id'] for x in bos[db_id]]
    check = [True if bo_id in bo_ids else False for bo_id in retrieved]
    assert all(check), f'{db_id} {sample_id}'

AssertionError: address 5082

### valid_bo

In [36]:
class ARGS():
    exp_name = 'pipeline_exp'
    ds = 'bird'
    type = 'dev'
    task = 'fill_in'
    with_bos = True
    
args = ARGS()

description_file = f'description.json' if args.ds == 'spider' else f'{args.ds}_description.json'
experiment_folder = proj_path / args.exp_name / args.ds

description_file = 'bird_description.json' if args.ds == 'bird' else 'description.json'
tables, *_ = load_raw_data(proj_path / 'data' / args.ds, load_test=False)
with (proj_path / 'data' / description_file).open() as f:
    all_descriptions = json.load(f)
tables = process_all_tables(tables, descriptions=all_descriptions)

samples = load_samples_spider_bird(proj_path / 'data' / f'{args.ds}_{args.type}.json')

In [37]:
SKIP_DB_IDS = ['movie_platform']
eval_path = experiment_folder / 'evals'

no_bos_path = eval_path / f'result-no_bos-{args.type}.csv'
with_bos_path = eval_path / f'result-with_bos-{args.type}.csv'
assert no_bos_path.exists(), f'Run with the `task=evaluate, type={args.type}` first'
assert with_bos_path.exists(), f'Run with the `task=evaluate, type={args.type}` first'

df_no_bos = pd.read_csv(no_bos_path)
df_no_bos = df_no_bos[~df_no_bos['db_id'].isin(SKIP_DB_IDS)]
df_no_bos.reset_index(drop=True, inplace=True)
df_no_bos.drop(columns=['retrieved'], inplace=True)
df_with_bos = pd.read_csv(with_bos_path)
df_cates = df_no_bos.groupby('db_id')['target_complexity'].apply(_get_categories).rename('category').apply(_format_interval)
df_no_bos = pd.merge(df_no_bos, df_cates.reset_index('db_id', drop=True), left_index=True, right_index=True)
df = pd.merge(
    left=df_with_bos,
    right=df_no_bos,
    how='inner',
    on=['db_id'],
    suffixes=('_bo', '')
)

In [43]:
sample_id_retrieved = df.groupby(['db_id', 'sample_id'])['retrieved'].apply(set).reset_index()
for i, row in sample_id_retrieved.iterrows():
    db_id = row['db_id']
    sample_id = row['sample_id']
    retrieved = row['retrieved']
    bos_ids = [x['sample_id'] for x in bos[db_id]]
    check = [True if bo_id in bos_ids else False for bo_id in retrieved]
    assert all(check), f'{db_id} {sample_id}'

AssertionError: address 5082

In [38]:
np.setdiff1d(df_with_bos['db_id'], df['db_id'])

array([], dtype=object)

In [39]:
group_column = ['db_id', 'retrieved']
# count the exec result with/without BOs
# > 0 means improved with BOs
# = 0 means execution is same with/witout BOs
# < 0 means worse with BOs
execution_improvement = df.groupby(group_column)[['exec_res', 'exec_res_bo']].sum().diff(axis=1)['exec_res_bo'].rename('execution_improvement')

# no_bos: tsed between (source=pred_sql, target=target_sql)
# with_bos: tsed between (source=pred_sql, target=target_sql)
# merit = tsed(pred_sql_bo, target_sql) - tsed(pred_sql, target_sql)
# merit > 0 means similarity to the targer_sql improved with BOs
# merit = 0 means similarity to the target_sql is same with/without BOs
# merit < 0 means similarity to the target_sql getting worse with BOs
merit_structural = df.groupby(group_column)[['structural_score', 'structural_score_bo']].mean().diff(axis=1)['structural_score_bo'].rename('merit_structural')
merit_semantic = df.groupby(group_column)[['semantic_score', 'semantic_score_bo']].mean().diff(axis=1)['semantic_score_bo'].rename('merit_semantic')
merit = df.groupby(group_column)[['f1_score', 'f1_score_bo']].mean().diff(axis=1)['f1_score_bo'].rename('merit')

In [40]:
ranks = merit.reset_index().groupby(['db_id'])['merit'].rank(method='first', ascending=False).rename('rank').astype(np.int64)
merit = pd.concat([merit.reset_index(), ranks], axis=1)
merit_by_rank = merit.sort_values(by=['db_id', 'rank'], ascending=True)
# merit_by_rank.to_csv(experiment_folder / 'evals' / f'merits.csv', index=False)

In [41]:
n_bos_select = 25

bo_path = experiment_folder / 'predictions' / 'create_bo' / f'final_{args.ds}_train_bo.json'
with bo_path.open() as f:
    bos = json.load(f)

bos_ids_by_db_id = defaultdict(set)
for db_id, xs in bos.items():
    bos_ids_by_db_id[db_id].update({x['sample_id'] for x in xs})

test_bos = defaultdict(list)
for x in merit_by_rank.loc[:, ['db_id', 'retrieved']].to_dict(orient='records'):
    bo_id = x['retrieved']
    db_id = x['db_id']
    if len(test_bos[db_id]) >= n_bos_select:
        continue
    
    assert bo_id in bos_ids_by_db_id[db_id], f'{bo_id} not in {db_id}'
    bo = list(filter(lambda x: x['sample_id'] == bo_id, bos[db_id]))[0]
    test_bos[db_id].append(bo)

AssertionError: 3058 not in address

### Analysis

# Similarity between dataset

In [ ]:
def get_parsed_sql(samples: dict, tables: dict):
    error_ids = []
    parsed = defaultdict(dict)
    iterator = tqdm(samples, total=len(samples))
    for sample in iterator:
        db_id = sample.db_id
        sample_id = sample.sample_id
        iterator.set_description(f"{db_id}")
        schema = Schema(tables[db_id].db_schema)
        sql_i = sample.final.sql
        try:
            ei = extract_all(sql_i, schema)
            assert len(ei['sel']) > 0, f'No selection found-{db_id}-{sample_id}'
        except Exception as e:
            error_ids.append((db_id, sample_id, str(e)))
            parsed[db_id].append(None)
            continue
        parsed[db_id][sample_id] = ei
    return parsed, error_ids

train_parsed, error_ids = get_parsed_sql(train_samples, bird_tables)
dev_parsed, error_ids = get_parsed_sql(dev_samples, bird_tables)
test_parsed, error_ids = get_parsed_sql(test_samples, bird_tables)

In [ ]:
# with open(proj_path / 'data' / 'pkl_files' / 'bird_train_parsed.pkl', 'wb') as f:
#     pickle.dump(train_parsed, f)

# with open(proj_path / 'data' / 'pkl_files' / 'bird_dev_parsed.pkl', 'wb') as f:
#     pickle.dump(dev_parsed, f)

# with open(proj_path / 'data' / 'pkl_files' / 'bird_test_parsed.pkl', 'wb') as f:
#     pickle.dump(test_parsed, f)

with open(proj_path / 'data' / 'pkl_files' / 'bird_dev_parsed.pkl', 'rb') as f:
    dev_parsed = pickle.load(f)

with open(proj_path / 'data' / 'pkl_files' / 'bird_test_parsed.pkl', 'rb') as f:
    test_parsed = pickle.load(f)

In [ ]:
from itertools import combinations, product
from collections import defaultdict
from src.eval_utils import get_all_partial_score

def measure_inter_score(parsed1: dict[str, tuple], parsed2: dict[str, tuple]):
    results = defaultdict()
    assert len(parsed1) == len(parsed2), f"Length mismatch-1: {len(parsed1)} 2:{len(parsed2)}"
    db_ids = list(parsed1.keys())
    for db_id in db_ids:
        o1 = parsed1[db_id]
        o2 = parsed2[db_id]
        n1 = len(o1)
        n2 = len(o2)
        semantic_sim = np.zeros((n1, n2), dtype=np.float32)
        structural_sim = np.zeros((n1, n2), dtype=np.float32)
        overall_sim = np.zeros((n1, n2), dtype=np.float32)

        idxs = list(product(range(n1), range(n2)))
        iterator = tqdm(idxs, total=len(idxs), desc=f"{db_id}")
        for i, j in iterator:
            ei = o1[i]
            ej = o2[j]

            _, final_score = get_all_partial_score(ei, ej, use_bert=True)

            structural_sim[i, j] = final_score['structural']
            semantic_sim[i, j] = final_score['semantic']
            overall_sim[i, j] = final_score['overall']

        results[db_id] = {
            'semantic': semantic_sim,
            'struct': structural_sim,
            'overall': overall_sim
        }
    return results

results = measure_inter_score(dev_parsed, test_parsed)
with (proj_path / 'data' / 'pkl_files' / 'bird_dev_test_similarity.pkl').open('wb') as f:
    pickle.dump(results, f)

# Complexity between datasets

In [ ]:
def measure_complexity(samples, tables):
    cs = []
    for s in tqdm(samples, total=len(samples)):
        schema = Schema(tables[s.db_id].db_schema)
        output = extract_all(s.final.sql, schema)
        complexity = get_complexity(output)
        cs.append(complexity)
    return cs

train_complexities = measure_complexity(train_samples, bird_tables)
dev_complexities = measure_complexity(dev_samples, bird_tables)
test_complexities = measure_complexity(test_samples, bird_tables)

In [ ]:
for c, n in zip([train_complexities, dev_complexities, test_complexities], ['train', 'dev  ', 'test ']):
    print(f'[{n}] Mean={np.mean(c):.4f} +/-{np.std(c):.4f}, Median={np.median(c):.4f}')

In [ ]:
stats = defaultdict(list)
for s in dev_samples:
    stats[s.db_id].append(s)